# Data Analysis A Bayesian Tutorial

working out of Second Edition by D.S. Sivia


Baye's Theorem: 
$$\begin{equation}
\label{eq:bayes}
P(\theta|\textbf{D},I) = P(D|\theta,I ) \frac{P(\theta| I)}{P(\textbf{D}| I)}
\end{equation}$$
Where $\theta$ is a parameter in a model described by I and D is the data.

In the case of beginning with a set of data points $(x_i,y_i\pm \sigma_y)$, where the uncertainty on y is a constant for each data point. y_i also has some error in the measurement, desrcibed by a Gaussian around the true value $\hat{y_i}$ and the uncertainty on $\sigma_y$.

In other words, $y_i=\hat{y_i}+\epsilon$ and $\epsilon$ follows a Normal distribution about $\hat{y_i}$.

It is known that the relationship between the true values that we are trying to measure is $\hat{y_i}=f(\hat{x_i})=a\hat{x_i}+b.$ a and b are the parameters that I will try to estimate using Baye's theorem. 

The probability of getting a value $y_i$ given $x_i$,a, and b is then given by a Gaussian:

$P(y_i|x_i,a,b,I)=N(ax_i+b,\sigma_y)$

If I assume a flat prior (no inital preference for the values of a and b) since I have not seen the data, the prior term can be neglected. Further, if I am estimating parameters and do not care about the true value of $P(\theta|\textbf{D},I)$ then the evidence can also be neglected. 

This yields $P(\theta|\textbf{D},I) = P(D|\theta,I )$
This is the beauty of Baye's theorem since it will be easier to determine the evidence than the posterior.

For each data point k in the N amount of data the likelihood can be calculated. For the overall likelihood these can be multiplied. 
Solving analytically, one can find that 
$P(\theta|\textbf{D},I) = P(D|\theta,I )=\prod_{k=1}^{N}\frac{1}{\sigma_y\sqrt{2\pi}}\exp(\frac{-(y_k-(ax_k+b))^2}{2\sigma_y})$

Now that I know my probability distribution for a given data, I simply need to find the a and b parameters in my model that make this most likely. This turns it into a $\chi^2$ minimization problem. If I take the logarithm of P I get:
$L(\theta|\textbf{D},I) = L(D|\theta,I )=\sum_{k=1}^{N}constant+(\frac{-(y_k-(ax_k+b))^2}{2\sigma_y})=\sum_{k=1}^{N}constant+\chi^2$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#set parameters, arrays
a=2
b=3
sig_y=0.05
data_len=10
y_i=np.zeros(data_len)
x_i=np.zeros(data_len)

def y_hat(x_i):
    return(a*x_i+b)

pi=np.pi
e=np.e

In [ ]:
#create my Gaussian
def gauss(x,y_data,sig_y,a,b):
    return(1/(sig_y*np.sqrt(2*pi))*e**(-(y_data-(a*x+b))**2/(2*sig_y)))

In [ ]:
#what does this look like?
y_data=np.linspace(0,a*15)
x_test=5
g=gauss(x_test,y_data, sig_y,a,b)
plt.plot(y_data,g)
plt.xlabel("y value for given x")
plt.ylabel("probability")
print("mean is", a*x_test+b)


So now I have a gaussian that my noise for each data point should follow. For each point $x_k,y_k$ I will know $x_i$ but y_i will be drawn from this gaussian. 

In [ ]:
#create my x array, going to choose even spacing
x_i=np.arange(0,data_len)

In [ ]:
from numpy import random
for i in range(len(x_i)):
    mean=a*x_i[i]+b
    sigma=sig_y

    s = np.random.normal(mean, sigma, 1)
    y_i[i]=s
    
#print(y_i)



In [ ]:
plt.plot(x_i,y_hat(x_i), label="true y value")
plt.errorbar(x_i,y_i, yerr=sig_y, fmt='o', label="measured value")
plt.legend()
plt.ylabel("y")
plt.xlabel("x")

#now I need to fit for my unkown a and b parameter (this assumes I did not know a and b that generated my data)
#essentially I am going to be performing a chi^2 test
#this can be done analytically for this simple problem by taking derivatives of p
#here I am showing a numerical solution

In [ ]:
a_guess=np.linspace(a*0.8,a*1.2,500)
b_guess=np.linspace(b*0.8,b*1.2,400)

aa,bb=np.meshgrid(a_guess,b_guess)
chi2vals = np.zeros(np.prod(aa.shape))


def chi2(a_val, b_val,x_i,y_i,sig_y):
    val=np.sum(((y_i-(a_val*x_i+b_val))/(sig_y))**2 )
    return val

# for i in range(len(a_guess)-1):
#     for j in range(len(b_guess)-1):
#         chi2valstest[i][j]=chi2(a_guess[i],b_guess[j],x_i,y_i,sig_y)

for i,(ai,bi) in enumerate(zip(aa.flatten(),bb.flatten())):
    chi2vals[i]=chi2(ai,bi,x_i,y_i,sig_y)
    


In [ ]:
# cp=ax.contour(a_guess,b_guess, chi2vals)

# ax.clabel(cp, inline=True, 
#           fontsize=10)


cp = plt.contourf(aa, bb, np.exp(-0.5*chi2vals.reshape(aa.shape)), levels = 1000)
plt.colorbar(cp)
#plt.imshow(a_guess, b_guess, chi2vals, cmap='hot')
# import seaborn as sns
#ax=sns.heatmap(chi2vals)
plt.scatter(a,b, marker='+', c='r', lw = 0.9)
plt.xlabel("a")
plt.ylabel("b")
# plt.title("$\chi^2$ values for a and b")

So, as we can see $\chi^2$ is minimized at the correct value of a and b, but this minimization occurs over a very long range. Next, I will test how our uncertainty effects this. If $\sigma_y$ is smaller will our parameters be more well constrained?

Surprisingly (are you really *that* surprised? -Ste-), this did not improve our constraint on a and b. 

I may have errors in this code.  (you definitely have. everyone has)


In [ ]:
#after this point still working on 

In [ ]:
# #code from Walter Del Pozzo
# def FindHeightForLevel(inArr, adLevels):
#    """
#    Computes the height of a :math:`2D` function for given levels

#    :param inArr: function values
#    :type inArr: array
#    :param adLevels: levels
#    :type adLevels: list or array

#    :return: function values with levels closest to *levels*
#    :rtype: array
#    """

#    # flatten the array
#    oldshape = np.shape(inArr)
#    adInput  = np.reshape(inArr,oldshape[0]*oldshape[1])
#    #adInput=inArr

#    # get array specifics
#    nLength  = np.size(adInput)

#    # create reversed sorted list
#    adTemp   = -1.0 * adInput
#    adSorted = np.sort(adTemp)
#    adSorted = -1.0 * adSorted

#    # create the normalised cumulative distribution
#    adCum    = np.zeros(nLength)
#    adCum[0] = adSorted[0]

#    for i in range(1,nLength):
#        adCum[i] = np.logaddexp(adCum[i-1], adSorted[i])

#    adCum    = adCum - adCum[-1]

#    # find the values closest to levels
#    adHeights = []
#    for item in adLevels:
#        idx = (np.abs(adCum-np.log(item))).argmin()
#        adHeights.append(adSorted[idx])

#    adHeights = np.array(adHeights)
#    return np.sort(adHeights)

In [ ]:
#test

In [ ]:
from credibleregions import FindHeightForLevel as FHFL

In [ ]:
#FHFL?

In [ ]:
levelval=(FHFL(np.log(chi2vals.reshape(aa.shape)),[0.9]))
print(levelval)
levelval=(levelval[0])
print(levelval)


In [ ]:
#print(chi2vals.reshape(aa.shape))
chi2matrix=chi2vals.copy()
chi2matrix=chi2matrix.reshape(aa.shape)
print(chi2matrix)
# print(len(chi2matrix[:,0]))
# print(aa.shape)

In [ ]:


def selectrange(matrix1):
    matrix=np.log(matrix1.copy())
    for j in range(len(matrix[:,0])):
        for k in range(len(matrix[0,:])):
            if matrix[j][k]<=levelval:
                matrix[j][k]=1
                #print(j,k)
            else:
                matrix[j][k]=0
    return matrix

curvematrix=selectrange(chi2matrix)
            
curve90=plt.contourf(aa,bb, curvematrix)
plt.colorbar(curve90)
#plt.legend()
plt.scatter(a,b, marker='+', c='r', lw = 0.9)
plt.xlabel("a")
plt.ylabel("b")
#plt.colorbar(cp)


#print(chi2matrix,"and", curvematrix)

In [ ]:

# import numpy as np

# #code from Walter Del Pozzo
# def FindHeightForLevel(inArr, adLevels):
#    """
#    Computes the height of a :math:`2D` function for given levels

#    :param inArr: function values
#    :type inArr: array
#    :param adLevels: levels
#    :type adLevels: list or array

#    :return: function values with levels closest to *levels*
#    :rtype: array
#    """

#    # flatten the array
#    oldshape = np.shape(inArr)
#    adInput  = np.reshape(inArr,oldshape[0]*oldshape[1])
#    #adInput=inArr

#    # get array specifics
#    nLength  = np.size(adInput)

#    # create reversed sorted list
#    adTemp   = -1.0 * adInput
#    adSorted = np.sort(adTemp)
#    adSorted = -1.0 * adSorted

#    # create the normalised cumulative distribution
#    adCum    = np.zeros(nLength)
#    adCum[0] = adSorted[0]

#    for i in range(1,nLength):
#        adCum[i] = np.logaddexp(adCum[i-1], adSorted[i])

#    adCum    = adCum - adCum[-1]

#    # find the values closest to levels
#    adHeights = []
#    for item in adLevels:
#        print(item) 
#        idx = (np.abs(adCum-np.log(item))).argmin()
#        adHeights.append(adSorted[idx])

#    adHeights = np.array(adHeights)
#    return np.sort(adHeights), idx, adCum, adSorted, adCum[-1]

# FindHeightForLevel(np.log(chi2vals.reshape(aa.shape)),[0.5, 0.9])